In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import seaborn as sns
import scipy as sp
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
    
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [4]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 11s, sys: 5.36 s, total: 1min 16s
Wall time: 1min 16s


In [5]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]

In [6]:
assess_title = ['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)']
def remove_index_calc(df):
    additional_remove_index = []
    for i, session in df.groupby('installation_id', sort=False):
        last_row = session.index[-1]
        session = session[session.title.isin(assess_title)]
        first_row = session.index[-1] + 1
        for j in range(first_row, last_row+1):
            additional_remove_index.append(j)                
    return additional_remove_index
additional_remove_index = remove_index_calc(train)
train = train[~train.index.isin(additional_remove_index)]

# Preprocess and Feature engineering

In [7]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique())))
    
    list_of_eventid = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))

    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
                
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_eventid

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_eventid = encode_title(train, test)

CPU times: user 38.8 s, sys: 2.99 s, total: 41.8 s
Wall time: 41.5 s


In [8]:
def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    
    last_session_time_sec = 0
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    miss = 0
    crys_game_true = 0; crys_game_false = 0; total_crys_game_true = 0; total_crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0; total_tree_game_true = 0; total_tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0; total_magma_game_true = 0; total_magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    t_crys_game_acc = []; t_tree_game_acc = []; t_magma_game_acc = []
    crys_act_true = 0; crys_act_false = 0; total_crys_act_true = 0; total_crys_act_false = 0
    tree_act_true = 0; tree_act_false = 0; total_tree_act_true = 0; total_tree_act_false = 0
    magma_act_true = 0; magma_act_false = 0; total_magma_act_true = 0; total_magma_act_false = 0
    crys_act_acc = []; tree_act_acc = []; magma_act_acc = []
    t_crys_act_acc = []; t_tree_act_acc = []; t_magma_act_acc = []
        
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent   
            
            if session_type == "Game":
                true = session['event_data'].str.contains('true').sum()
                false = session['event_data'].str.contains('false').sum() 
                if session_world == activities_world["CRYSTALCAVES"]:
                    crys_game_true += true
                    crys_game_false += false
                    total_crys_game_true += true
                    total_crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    t_crys_game_acc += crys_game_acc
                elif session_world == activities_world["TREETOPCITY"]:
                    tree_game_true += true
                    tree_game_false += false
                    total_tree_game_true += true
                    total_tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    t_tree_game_acc += tree_game_acc
                elif session_world == activities_world["MAGMAPEAK"]:
                    magma_game_true += true
                    magma_game_false += false
                    total_magma_game_true += true
                    total_magma_game_false += false
                    magma_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    t_magma_game_acc += magma_game_acc
                else:
                    pass
                
            #if session_type == "Activity":
            #    true = session['event_data'].str.contains('true').sum()
            #    false = session['event_data'].str.contains('false').sum() 
            #    if session_world == activities_world["CRYSTALCAVES"]:
            #        crys_act_true += true
            #        crys_act_false += false
            #        total_crys_act_true += true
            #        total_crys_act_false += false
            #        crys_act_acc.append(true / (true + false) if (true + false) != 0 else 0)
            #        t_crys_act_acc += crys_act_acc
            #    elif session_world == activities_world["TREETOPCITY"]:
            #        tree_act_true += true
            #        tree_act_false += false
            #        total_tree_act_true += true
            #        total_tree_act_false += false
            #        tree_act_acc.append(true / (true + false) if (true + false) != 0 else 0)
            #        t_tree_act_acc += tree_act_acc
            #    elif session_world == activities_world["MAGMAPEAK"]:
            #        magma_act_true += true
            #        magma_act_false += false
            #        total_magma_act_true += true
            #        total_magma_act_false += false
            #        magma_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
            #        t_magma_act_acc += magma_act_acc
            #    else:
            #        pass

        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features.update(time_spent_each_act.copy()) # cumulative gameplay time in each title
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            if session_world == activities_world["CRYSTALCAVES"]:
                features["total_game_true"] = total_crys_game_true
                features["total_game_false"] = total_crys_game_false
                features['total_game_accuracy'] = total_crys_game_true / (total_crys_game_true + total_crys_game_false) if (total_crys_game_true + total_crys_game_false) != 0 else 0
                features["all_game_accuracy_std"] = np.std(t_crys_game_acc) if len(t_crys_game_acc) >=1 else 0
                features["game_true_bet_assess"] = crys_game_true
                features["game_false_bet_assess"] = crys_game_false
                features['game_accuracy_bet_assess'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std_bet_assess"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            elif session_world == activities_world["TREETOPCITY"]:
                features["total_game_true"] = total_tree_game_true
                features["total_game_false"] = total_tree_game_false
                features['total_game_accuracy'] = total_tree_game_true / (total_tree_game_true + total_tree_game_false) if (total_tree_game_true + total_tree_game_false) != 0 else 0
                features["all_game_accuracy_std"] = np.std(t_tree_game_acc) if len(t_tree_game_acc) >=1 else 0
                features["game_true_bet_assess"] = tree_game_true
                features["game_false_bet_assess"] = tree_game_false
                features['game_accuracy_bet_assess'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            elif session_world == activities_world["MAGMAPEAK"]:
                features["total_game_true"] = total_magma_game_true
                features["total_game_false"] = total_magma_game_false
                features['total_game_accuracy'] = total_magma_game_true / (total_magma_game_true + total_magma_game_false) if (total_magma_game_true + total_magma_game_false) != 0 else 0
                features["all_game_accuracy_std"] = np.std(t_magma_game_acc) if len(t_magma_game_acc) >=1 else 0
                features["game_true_bet_assess"] = magma_game_true
                features["game_false_bet_assess"] = magma_game_false
                features['game_accuracy_bet_assess'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            crys_game_true = 0
            crys_game_false = 0
            tree_game_true = 0
            tree_game_false = 0
            magma_game_true = 0
            magma_game_false = 0
            crys_game_acc = []
            tree_game_acc = []
            magma_game_acc = []
                
            #if session_world == activities_world["CRYSTALCAVES"]:
            #    features["total_act_true"] = total_crys_act_true
            #    features["total_act_false"] = total_crys_act_false
            #    features['total_act_accuracy'] = total_crys_act_true / (total_crys_act_true + total_crys_act_false) if (total_crys_act_true + total_crys_act_false) != 0 else 0
            #    features["all_act_accuracy_std"] = np.std(t_crys_act_acc) if len(t_crys_act_acc) >=1 else 0
            #    features["act_true_bet_assess"] = crys_act_true
            #    features["act_false_bet_assess"] = crys_act_false
            #    features['act_accuracy_bet_assess'] = crys_act_true / (crys_act_true + crys_act_false) if (crys_act_true + crys_act_false) != 0 else 0
            #    features["act_accuracy_std"] = np.std(crys_act_acc) if len(crys_act_acc) >=1 else 0
            #    features["last_act_acc"] = crys_act_acc[-1] if len(crys_act_acc) >=1 else 0
            #elif session_world == activities_world["TREETOPCITY"]:
            #    features["total_act_true"] = total_tree_act_true
            #    features["total_act_false"] = total_tree_act_false
            #    features['total_act_accuracy'] = total_tree_act_true / (total_tree_act_true + total_tree_act_false) if (total_tree_act_true + total_tree_act_false) != 0 else 0
            #    features["all_act_accuracy_std"] = np.std(t_tree_act_acc) if len(t_tree_act_acc) >=1 else 0
            #    features["act_true_bet_assess"] = tree_act_true
            #    features["act_false_bet_assess"] = tree_act_false
            #    features['act_accuracy_bet_assess'] = tree_act_true / (tree_act_true + tree_act_false) if (tree_act_true + tree_act_false) != 0 else 0
            #    features["act_accuracy_std"] = np.std(tree_act_acc) if len(tree_act_acc) >=1 else 0
            #    features["last_act_acc"] = tree_act_acc[-1] if len(tree_act_acc) >=1 else 0
            #elif session_world == activities_world["MAGMAPEAK"]:
            #    features["total_act_true"] = total_magma_act_true
            #    features["total_act_false"] = total_magma_act_false
            #    features['total_act_accuracy'] = total_magma_act_true / (total_magma_act_true + total_magma_act_false) if (total_magma_act_true + total_magma_act_false) != 0 else 0
            #    features["all_act_accuracy_std"] = np.std(t_magma_act_acc) if len(t_magma_act_acc) >=1 else 0
            #    features["act_true_bet_assess"] = magma_act_true
            #    features["act_false_bet_assess"] = magma_act_false
            #    features['act_accuracy_bet_assess'] = magma_act_true / (magma_act_true + magma_act_false) if (magma_act_true + magma_act_false) != 0 else 0
            #    features["act_accuracy_std"] = np.std(magma_act_acc) if len(magma_act_acc) >=1 else 0
            #    features["last_act_acc"] = magma_act_acc[-1] if len(magma_act_acc) >=1 else 0
            # crys_act_true = 0
            # crys_act_false = 0
            # tree_act_true = 0
            # tree_act_false = 0
            # magma_act_true = 0
            # magma_act_false = 0
            # crys_act_acc = []
            # tree_act_acc = []
            # magma_act_acc = []
            
            # from the end of previous assessment to the start of next assessment --------------------------
                        
            # unique type --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            
            # nums in target assessment data ------------------------------------------
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
                features['duration_std'] = 0
                features['last_duration'] = 0
                features['duration_max'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
                features['last_duration'] = durations[-1]
                features['duration_max'] = np.max(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds) 
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        miss += np.sum(session["misses"])
            
        user_world_count["world_"+str(session_world)] += session.shape[0]

        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [9]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)


(17690, 475)


In [10]:
del train

In [11]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)


(1000, 475)


In [12]:
del test

# Feature selection and modelling

In [13]:
#correlations = new_train.corr().abs()
#correlations = correlations.mask(np.tril(np.ones(correlations.shape)).astype(np.bool))
#correlations = correlations.stack().reset_index()
#corr_columns = ["level_0", "level_1", "value"]
#correlations.columns = corr_columns
#correlations = correlations.sort_values("value", ascending=False).reset_index(drop=True)

#high_corr = correlations[correlations["value"] == 1]

#high_corr_features = []
#for i in range(high_corr.shape[0]):
#    if high_corr.iloc[i]["level_0"] not in high_corr_features and high_corr.iloc[i]["level_1"] not in high_corr_features:
#        high_corr_features.append(high_corr.iloc[i]["level_0"])
#    elif high_corr.iloc[i]["level_0"] in high_corr_features and high_corr.iloc[i]["level_1"] not in high_corr_features:
#        high_corr_features.append(high_corr.iloc[i]["level_1"])
#    elif high_corr.iloc[i]["level_0"] not in high_corr_features and high_corr.iloc[i]["level_1"] in high_corr_features:
#        high_corr_features.append(high_corr.iloc[i]["level_0"])

In [14]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = []
for i in X_train.columns:
    if X_train[i].std() == 0 and i not in remove_features:
        remove_features.append(i)
X_train = X_train.drop(remove_features, axis=1)
X_train = X_train[sorted(X_train.columns.tolist())]
y_train = new_train.accuracy_group
print(X_train.shape)

(17690, 449)


In [15]:
n_folds=5
skf=StratifiedKFold(n_splits = n_folds)
coefficients = []
models = []
train_qwk_scores = []
test_qwk_scores = []
lgbm_params = {
    "objective" : "regression",
    "metric" : "rmse",
    "tree_learner": "serial",
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}

features_list = [i for i in X_train.columns if i != "installation_id"]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    optR = OptimizedRounder()
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_train2 = X_train2.drop(['installation_id'],axis=1)
    
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    test2 = pd.concat([X_test2, y_test2], axis=1)
    test2 = test2.groupby('installation_id').apply(lambda x: x.sample(1, random_state=1223)).reset_index(drop=True)
    X_test2 = test2.drop(["accuracy_group", "installation_id"], axis=1)
    y_test2 = test2["accuracy_group"]
    
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        num_boost_round=100000,
        early_stopping_rounds=10,
    )
    
    models.append(clf)
    train_predict = clf.predict(X_train2, num_iteration = clf.best_iteration)
    test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
    
    optR.fit(train_predict.reshape(-1,), y_train2)
    tmp_coefficients = optR.coefficients()
    print("fold_"+str(i)+" coefficients: ", tmp_coefficients)
    opt_train_preds = optR.predict(train_predict.reshape(-1, ), tmp_coefficients)
    train_qwk_score = qwk(y_train2, opt_train_preds)
    print("training qwk: ", train_qwk_score)
    opt_test_preds = optR.predict(test_predict.reshape(-1, ), tmp_coefficients)
    test_qwk_score = qwk(y_test2, opt_test_preds)
    print("validation qwk: ", test_qwk_score)
    train_qwk_scores.append(train_qwk_score)
    test_qwk_scores.append(test_qwk_score)
    coefficients.append(tmp_coefficients)
    
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

print("                             ")
print("-----------------------------")
print('coefficients: ', coefficients)
print('train qwk list:', train_qwk_scores)
print('train qwk average score:',np.mean(train_qwk_scores))
print('valid qwk list: ', test_qwk_scores)
print('valid qwk average score:',np.mean(test_qwk_scores))

[1]	training's rmse: 1.25321	valid_1's rmse: 1.27337
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 1.24972	valid_1's rmse: 1.27031
[3]	training's rmse: 1.24628	valid_1's rmse: 1.26729
[4]	training's rmse: 1.24291	valid_1's rmse: 1.2644
[5]	training's rmse: 1.23959	valid_1's rmse: 1.2615
[6]	training's rmse: 1.23633	valid_1's rmse: 1.25871
[7]	training's rmse: 1.23312	valid_1's rmse: 1.25592
[8]	training's rmse: 1.22997	valid_1's rmse: 1.25323
[9]	training's rmse: 1.22687	valid_1's rmse: 1.25053
[10]	training's rmse: 1.22383	valid_1's rmse: 1.24794
[11]	training's rmse: 1.22084	valid_1's rmse: 1.24532
[12]	training's rmse: 1.21791	valid_1's rmse: 1.24283
[13]	training's rmse: 1.21514	valid_1's rmse: 1.24041
[14]	training's rmse: 1.21242	valid_1's rmse: 1.23801
[15]	training's rmse: 1.20963	valid_1's rmse: 1.23556
[16]	training's rmse: 1.20701	valid_1's rmse: 1.23326
[17]	training's rmse: 1.20444	valid_1's rmse: 1.23101
[18]	training's rmse: 1.20174	va

- random state 1223
- train qwk list: [0.62739484, 0.63408544, 0.64210887, 0.62687109, 0.63047162]
- train qwk average score: 0.6321863719999999
- valid qwk list:  [0.54421832, 0.54740015, 0.54092954, 0.48252857, 0.50422872]
- valid qwk average score: 0.52386106

In [16]:
feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True).head(500)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,session_title,886,875,917,1011,911,920.0,48.066620,0.052246
1,accumulated_accuracy_group,526,566,590,674,641,599.4,52.701423,0.087924
2,Clip,230,258,201,289,272,250.0,31.208973,0.124836
3,misses,162,204,228,266,267,225.4,39.656525,0.175938
4,Chow Time_4070,223,204,217,220,134,199.6,33.434114,0.167506
5,last_game_acc,177,181,193,234,159,188.8,25.095019,0.132919
6,total_game_true,193,182,86,196,257,182.8,55.054155,0.301172
7,game_accuracy_std,168,118,181,200,159,165.2,27.330569,0.165439
8,total_game_accuracy,128,82,107,193,198,141.6,46.383618,0.327568
9,Chest Sorter (Assessment)_3021,125,156,123,137,157,139.6,14.609586,0.104653


In [17]:
print(X_train.shape)
print("")
print(", ".join('%s'%x for x in X_train.columns))

(17690, 449)

0, 1, 12 Monkeys_2000, 2, 3, Activity, Air Show, Air Show_2000, Air Show_2020, Air Show_2030, Air Show_2060, Air Show_2070, Air Show_2075, Air Show_3010, Air Show_3020, Air Show_3021, Air Show_3110, Air Show_3120, Air Show_3121, Air Show_4010, Air Show_4020, Air Show_4070, Air Show_4090, Air Show_4100, Air Show_4110, All Star Sorting, All Star Sorting_2000, All Star Sorting_2020, All Star Sorting_2025, All Star Sorting_2030, All Star Sorting_3010, All Star Sorting_3020, All Star Sorting_3021, All Star Sorting_3110, All Star Sorting_3120, All Star Sorting_3121, All Star Sorting_4010, All Star Sorting_4020, All Star Sorting_4030, All Star Sorting_4035, All Star Sorting_4070, All Star Sorting_4080, All Star Sorting_4090, All Star Sorting_4095, Assessment, Balancing Act_2000, Bird Measurer (Assessment)_2000, Bird Measurer (Assessment)_2010, Bird Measurer (Assessment)_2020, Bird Measurer (Assessment)_2030, Bird Measurer (Assessment)_3010, Bird Measurer (Assessment)_3020, Bird 

In [18]:
#import eli5
#from eli5.sklearn import PermutationImportance
#my_model = lgb.LGBMRegressor(**lgbm_params).fit(X_train2, y_train2)
#perm = PermutationImportance(my_model, random_state=1).fit(X_test2, y_test2)
#eli5.show_weights(perm, feature_names = X_test2.columns.tolist(), top = 500)

# prediction

In [19]:
X_test = new_test.drop(["installation_id", "accuracy_group"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
X_test = X_test[sorted(X_test.columns.tolist())]
pred_value = np.zeros([X_test.shape[0]])
test_coefficients = np.mean(coefficients, axis=0)
for model in models:
    pred_value += model.predict(X_test, num_iteration = model.best_iteration) /n_folds
test_pred_class= optR.predict(pred_value.reshape(-1, ), test_coefficients)
sample_submission["accuracy_group"] = test_pred_class
sample_submission.to_csv('submission.csv', index=False)

In [20]:
sample_submission["accuracy_group"].value_counts(normalize = True)

3    0.459
2    0.297
1    0.139
0    0.105
Name: accuracy_group, dtype: float64